# Import packages

In [ ]:
# # if using google drive
# %cd /content/drive/MyDrive/ff_repo/Multifirefly-Project

In [ ]:
from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

%load_ext autoreload
%autoreload 2


from data_wrangling import specific_utils, combine_info_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis.cluster_replacement import cluster_replacement_utils
from decision_making_analysis.decision_making import decision_making_class, decision_making_utils, intended_targets_classes
from decision_making_analysis.GUAT import GUAT_collect_info_class, GUAT_combine_info_class
from decision_making_analysis.compare_GUAT_and_TAFT import GUAT_vs_TAFT_class, GUAT_vs_TAFT_x_sessions_class, helper_GUAT_vs_TAFT_class
from visualization.matplotlib_tools import plot_trials, plot_behaviors_utils
from visualization.animation import animation_class
from null_behaviors import show_null_trajectory, find_best_arc, curvature_utils, curv_of_traj_utils
from machine_learning.ml_methods import regression_utils, classification_utils, prep_ml_data_utils, hyperparam_tuning_class
from visualization.plotly_polar_tools import plotly_utils_polar, plotly_for_ff_polar, plotly_for_trajectory_polar
from machine_learning.ml_methods import ml_methods_class
from machine_learning.ml_methods.advanced_ml_methods import advanced_regression_utils

import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from os.path import exists
import math
import copy
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import gc
from scipy import stats
from IPython.display import HTML
from matplotlib import rc
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
import os, sys, sys
from importlib import reload
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import (
    train_test_split, cross_val_score, cross_validate, KFold
)

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
pd.options.display.max_rows = 50



# Predict curvature

## explanatory_var

In [ ]:
from decision_making_analysis import free_selection, replacement, trajectory_info

In [ ]:
reload(free_selection)

In [ ]:
#raw_data_folder_path='all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330'
raw_data_folder_path='all_monkey_data/raw_monkey_data/monkey_Schro/data_0413'

moit = intended_targets_classes.ModelOfIntendedTargets(raw_data_folder_path=raw_data_folder_path, time_range_of_trajectory=[-0.5, 2.5], num_time_points_for_trajectory=10)
moit.get_monkey_data(already_retrieved_ok=True, include_ff_dataframe=True)
moit.ff_dataframe = moit.ff_dataframe[abs(moit.ff_dataframe['ff_angle_boundary']) <= math.pi/4]
moit.ff_dataframe = moit.ff_dataframe[moit.ff_dataframe['time_since_last_vis'] <= 2.5]

# moit.get_and_process_manual_anno_long(n_seconds_after_crossing_boundary=2.5)
moit.make_curvature_df([-25, 25], curv_of_traj_mode='distance')

moit.make_auto_annot()
moit.manual_anno = moit.auto_annot
moit.manual_anno_long = moit.auto_annot_long


moit.eliminate_crossing_boundary_cases(n_seconds_after_crossing_boundary=0.5)
moit.get_input_data(num_ff_per_row=5, select_every_nth_row=10, 
                    add_arc_info=False, curvature_df=moit.curvature_df, curv_of_traj_df=moit.curv_of_traj_df.copy()
                    )



# ML class

In [ ]:
window_for_curv_of_traj = [0, 50] # we want to predict into the future
curv_of_traj_mode = 'distance'
# get dependent variables
reload(curvature_utils)
curv_of_traj_df, traj_curv_descr = curv_of_traj_utils.find_curv_of_traj_df_based_on_curv_of_traj_mode(window_for_curv_of_traj, moit.monkey_information, 
                                                                                                      moit.ff_caught_T_new, curv_of_traj_mode=curv_of_traj_mode, truncate_curv_of_traj_by_time_of_capture=moit.truncate_curv_of_traj_by_time_of_capture)
all_point_index = moit.free_selection_x_df['point_index'].values
dependent_var = curv_of_traj_df.set_index('point_index').loc[all_point_index, 'curv_of_traj'].values

In [ ]:
ml_inst = ml_methods_class.MlMethods(x_var_df= moit.free_selection_x_df.drop(columns=['point_index'], errors='ignore'),
                                     y_var_df=curv_of_traj_df.set_index('point_index').loc[all_point_index, ['curv_of_traj']])

In [ ]:
ml_inst.x_var_df

In [ ]:
# ml_inst.use_ml_model_for_regression(ml_inst.x_var_df, ml_inst.y_var_df, model_names=['linreg', 'svr', 'dt', 'bagging', 'boosting', 'grad_boosting', 'rf'])
# ml_inst.model_comparison_df

## advanced ML models

In [ ]:
ml_inst.use_train_test_split(ml_inst.x_var_df, ml_inst.y_var_df)
df, best = advanced_regression_utils.use_advanced_model_for_regression(
    ml_inst.X_train, ml_inst.y_train, ml_inst.X_test, ml_inst.y_test,
    use_cv=False, cv_splits=5, verbose=True,
    save_dir=f"all_monkey_data/decision_making/{moit.monkey_name}/pred_curv_of_traj",
    resume=False
)

In [ ]:
ml_inst.use_train_test_split(ml_inst.x_var_df, ml_inst.y_var_df)
df, best = advanced_regression_utils.use_advanced_model_for_regression(
    ml_inst.X_train, ml_inst.y_train, ml_inst.X_test, ml_inst.y_test,
    use_cv=False, cv_splits=5, verbose=True,
    save_dir=f"all_monkey_data/decision_making/{moit.monkey_name}/pred_curv_of_traj",
    resume=False
)

In [ ]:
ml_inst.x_var_df.columns

In [ ]:
ml_inst.model_comparison_df, ml_inst.chosen_model_info = regression_utils.ml_model_for_regression(ml_inst.X_train, ml_inst.y_train, ml_inst.X_test, ml_inst.y_test,
                                                                                                )

# bagging prediction

In [ ]:
# X, y are your full dataset
model, metrics, _ = fit_plot_bagging_regression(
   ml_inst.x_var_df, ml_inst.y_var_df,
    test_size=0.2,
    random_state=123,
    n_estimators=300,
    max_samples=0.9,     # try 0.5–1.0
    max_features=1.0,    # try 0.5–1.0
    bootstrap=True,
)
print(metrics)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

def fit_plot_bagging_regression(
    X,
    y,
    test_size: float = 0.2,
    random_state: int = 42,
    base_estimator=None,
    n_estimators: int = 200,
    max_samples: float | int = 1.0,
    max_features: float | int = 1.0,
    bootstrap: bool = True,
    bootstrap_features: bool = False,
    n_jobs: int = -1,
):
    # Ensure arrays
    X_values = X.values if isinstance(X, pd.DataFrame) else np.asarray(X)
    y_values = (y.squeeze().values if isinstance(y, pd.DataFrame) else np.asarray(y)).ravel()

    X_train, X_test, y_train, y_test = train_test_split(
        X_values, y_values, test_size=test_size, random_state=random_state
    )

    if base_estimator is None:
        base_estimator = DecisionTreeRegressor(random_state=random_state)

    # sklearn compatibility: estimator (>=1.2) vs base_estimator (older)
    try:
        model = BaggingRegressor(
            estimator=base_estimator,
            n_estimators=n_estimators,
            max_samples=max_samples,
            max_features=max_features,
            bootstrap=bootstrap,
            bootstrap_features=bootstrap_features,
            random_state=random_state,
            n_jobs=n_jobs,
        )
    except TypeError:
        model = BaggingRegressor(
            base_estimator=base_estimator,
            n_estimators=n_estimators,
            max_samples=max_samples,
            max_features=max_features,
            bootstrap=bootstrap,
            bootstrap_features=bootstrap_features,
            random_state=random_state,
            n_jobs=n_jobs,
        )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Metrics (compat for older sklearn without `squared`)
    r2 = r2_score(y_test, y_pred)
    try:
        rmse = mean_squared_error(y_test, y_pred, squared=False)
    except TypeError:
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Plot Predicted vs Actual
    fig, ax = plt.subplots(figsize=(6.5, 6))
    ax.scatter(y_test, y_pred, alpha=0.6, edgecolor="none")

    vmin = float(min(np.min(y_test), np.min(y_pred)))
    vmax = float(max(np.max(y_test), np.max(y_pred)))
    ax.plot([vmin, vmax], [vmin, vmax], linestyle="--")

    ax.set_xlabel("Actual")
    ax.set_ylabel("Predicted")
    ax.set_title("Bagging Regression — Predicted vs Actual")

    ax.text(0.02, 0.98, f"R² = {r2:.3f}\nRMSE = {rmse:.4g}",
            transform=ax.transAxes, va="top", ha="left",
            bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="0.7"))

    plt.tight_layout()
    return model, {"r2": r2, "rmse": rmse}, (fig, ax)


# Feature importance

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Train model
rf = RandomForestRegressor(n_estimators=500, random_state=42)
rf.fit(ml_inst.x_var_df, ml_inst.y_var_df)

# Feature importances
importances = pd.Series(rf.feature_importances_, index=ml_inst.x_var_df.columns)
importances.sort_values(ascending=False).head(10)


In [ ]:
importances = pd.Series(rf.feature_importances_, index=ml_inst.x_var_df.columns).sort_values(ascending=False)
importances.head(10)

In [ ]:
importances

# What I used before

In [ ]:
stop!

In [ ]:
reload(pattern_by_trials)
reload(decision_making_class)
reload(decision_making_utils)
reload(show_null_trajectory)
reload(intended_targets_classes)
reload(regression_utils)
reload(curvature_utils)

## explanatory_var

In [ ]:
explanatory_var = moit.free_selection_x_df.drop(columns=['point_index'], errors='ignore').values

## dependent_var

In [ ]:
window_for_curv_of_traj = [0, 50] # we want to predict into the future
curv_of_traj_mode = 'distance'


curv_of_traj_df, traj_curv_descr = curv_of_traj_utils.find_curv_of_traj_df_based_on_curv_of_traj_mode(window_for_curv_of_traj, moit.monkey_information, 
                                                                                                      moit.ff_caught_T_new, curv_of_traj_mode=curv_of_traj_mode, truncate_curv_of_traj_by_time_of_capture=moit.truncate_curv_of_traj_by_time_of_capture)
all_point_index = moit.free_selection_x_df['point_index'].values
dependent_var = curv_of_traj_df.set_index('point_index').loc[all_point_index, 'curv_of_traj'].values

## train test split

In [ ]:
keep_whole_chunk = False
test_size = 0.2

scaler = StandardScaler()
dependent_var_sc = scaler.fit_transform(dependent_var.reshape(-1,1)) #scale data
explanatory_var_sc = scaler.fit_transform(explanatory_var)


if keep_whole_chunk:
    indices = np.arange(len(dependent_var_sc))
    num_test_points = int(len(indices)*test_size)
    num_train_points = len(indices)-num_test_points
    # make sure that the test chunk will be a whole segment in the data, to minimize the splitting up of the train and test chunks
    test_indice_start = np.random.randint(0, num_train_points)
    indices_test = indices[test_indice_start:test_indice_start+num_test_points]
    indices_train = np.setdiff1d(indices, indices_test)
    X_train = explanatory_var_sc[indices_train]
    X_test = explanatory_var_sc[indices_test]
    y_train = dependent_var_sc[indices_train]
    y_test = dependent_var_sc[indices_test]
else:
    X_train, X_test, y_train, y_test = train_test_split(explanatory_var_sc, dependent_var_sc, test_size=test_size, random_state=42)

In [ ]:
moit.free_selection_x_df.columns

In [ ]:
moit.free_selection_x_df

## ML

In [ ]:
X_train.shape

In [ ]:
STOP!

In [ ]:
# use MLPRegressor
mlp = MLPRegressor(random_state=1, max_iter=100).fit(X_train, y_train)
mlp.score(X_test, y_test)

In [ ]:
# use KNeighborsRegressor
n_neighbors = int(math.sqrt(len(y_train)))
neigh = KNeighborsRegressor(n_neighbors=n_neighbors)
neigh.fit(X_train, y_train)
neigh.score(X_test, y_test)

In [ ]:
# use RandomForestRegressor
regr = RandomForestRegressor(max_depth=10, random_state=0)
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

In [ ]:
# use linear regression
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_test, y_test)